In [2]:
import pandas as pd

# Load the Excel file
file_path = '새롬 판정 코드_LLM.xlsx'
df = pd.read_excel(file_path)

# Extract the unique names of "검사명"
unique_names = df['검사명'].unique()

# Display the unique names
unique_names_list = unique_names.tolist()
unique_names_list

['분변 얼리텍 대장암검사',
 '청력검사',
 '혈압검사',
 '안압검사',
 '말초혈액도말검사',
 '위내시경',
 '대장내시경',
 '상복부 초음파',
 '경동맥 초음파',
 '심장 초음파',
 '갑상선 초음파',
 '유방 초음파',
 '하복부 초음파',
 nan,
 '자궁경부 인유두종바이러스',
 '유방 X-선',
 '유방 X-선 + 유방 초음파',
 '흉부 X-선',
 '요추 CT',
 '뇌 CT',
 '요추 X-선',
 '경추 X-선',
 '저선량 폐 CT',
 '복부비만 CT',
 '뇌 MRI',
 '뇌 MRA',
 '경추 MRI',
 '요추 MRI',
 '좌측 무릎 X-선',
 '우측 무릎 X-선',
 '좌측 어깨 X-선',
 '우측 어깨 X-선',
 '경동맥 MRA',
 'C형 간염',
 '폐암검진',
 '자궁경부암',
 '심박 변이도',
 '심전도']

In [9]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback
from tqdm.auto import tqdm
from dotenv import load_dotenv

load_dotenv()

# Load the Excel file from the specified sheet
filepath = '새롬 판정 코드_LLM.xlsx'
df = pd.read_excel(filepath, sheet_name='소견정보-원본')

df = df.iloc[:586]
df = df.dropna(subset=['검사명'])

# Define and drop columns that are not needed
columns_to_drop = ['카테고리', '질환코드', '판정', '순번', '참조', '공단분류', '장비약어','사용유무','한글소견제목','영문소견제목','영문장비소견','외국어소견','추적관찰내용','구분1','구분2','구분3']
df = df.drop(columns=columns_to_drop)

# Ensure the new columns can accept string values
df['검사명(영문)'] = ''
df['소견명(영문)'] = ''
df['영문종합소견'] = ''

def content_translator(text):
    prompt_template = """
    Translate the content into English. Be aware that the content is medical information or terms. Only give the output.
    
    
    [medical content]
    {medical_content}
    """
    prompt = PromptTemplate.from_template(prompt_template)
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    output = chain.invoke({"medical_content": text})

    return output

new_filepath = "새롬 판정 코드_LLM_translated.xlsx"


# Populate new columns using tqdm for progress display
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    test_english = content_translator(row['검사명'])
    opinion_english = content_translator(row['소견명'])
    translated_content = content_translator(row['한글종합소견'])
    
    df.at[index, '검사명(영문)'] = test_english
    df.at[index, '소견명(영문)'] = opinion_english
    df.at[index, '영문종합소견'] = translated_content
    
    if (index + 1) % 100 == 0:
        df.to_excel(new_filepath, index=False)
        print("Excel file has been saved. row :", str(index + 1))

# Save the updated DataFrame to a new Excel file
df.to_excel(new_filepath, index=False)
print("New Excel file has been created:", new_filepath)

  0%|          | 0/442 [00:00<?, ?it/s]

Excel file has been saved. row : 100
Excel file has been saved. row : 400
Excel file has been saved. row : 500
New Excel file has been created: 새롬 판정 코드_LLM_translated.xlsx


In [1]:
import pandas as pd
from langchain_core.documents.base import Document

filepath = "새롬 판정 코드_LLM_translated.xlsx"
df_forloading = pd.read_excel(filepath,sheet_name='subjective')
# Function to create a Document from a DataFrame row
def create_document(row):
    return Document(page_content=row['영문종합소견'], metadata={'test': row['검사명'], 'korean': row['한글종합소견']})

# Sample DataFrame (please replace 'df' with your actual DataFrame)

# Convert each row in the DataFrame to a Document object using tqdm
docs = [create_document(row) for index, row in df_forloading.iterrows()]

print(docs)  # This line is just for verification; remove or replace as needed

KeyError: 'english_text'